In [60]:
import os
import cv2
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from skimage.feature import graycomatrix, graycoprops

# Pre-Processing

In [61]:
#Konversi gambar ke grayscale tanpa library
def convert_to_grayscale(image):
    grayscale_image = np.dot(image[..., :3], [0.2989, 0.5870, 0.1140])
    return grayscale_image.astype(np.uint8)

In [62]:
#Reduksi noise menggunakan mean filter
def apply_mean_filter(image):
    kernel = np.ones((3, 3)) / 9
    filtered_image = np.zeros_like(image)
    for i in range(1, image.shape[0] - 1):
        for j in range(1, image.shape[1] - 1):
            filtered_image[i, j] = np.sum(kernel * image[i-1:i+2, j-1:j+2])
    return filtered_image

In [63]:
#Peningkatan kontras mengunakan histogram equalization
def histogram_equalization(image):
    hist, bins = np.histogram(image.flatten(), 256, [0, 256])
    cdf = hist.cumsum()
    cdf_normalized = cdf * hist.max() / cdf.max()
    cdf = (cdf - cdf.min()) * 255 / (cdf.max() - cdf.min())
    cdf = cdf.astype('uint8')
    return cdf[image]

In [64]:
#Pipeline pre-processing gambar dengan membaca dimensi gambar secara dinamis
def preprocess_image_manual(image_path):
    # Membaca gambar menggunakan Pillow untuk mendapatkan dimensi
    with Image.open(image_path) as img:
        img = img.resize((224, 224))  # Resize gambar ke (224, 224) jika diperlukan
        image = np.array(img)  # Konversi ke numpy array (RGB)

    # Pastikan gambar memiliki 3 channel
    if image.ndim == 2:  # Jika grayscale
        image = np.stack([image] * 3, axis=-1)
    elif image.shape[2] == 4:  # Jika RGBA
        image = image[:, :, :3]

    # Pre-processing manual
    grayscale = convert_to_grayscale(image)
    noise_reduced = apply_mean_filter(grayscale)
    enhanced = histogram_equalization(noise_reduced)
    return enhanced

# Feature Extraction

In [65]:
#Ekstraksi fitur menggunakan GLCM dan Shape descriptors
def extract_features(image):
    # Texture features using GLCM
    glcm = graycomatrix(image, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    energy = graycoprops(glcm, 'energy')[0, 0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]

    # Shape features
    _, binary = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        cnt = contours[0]
        area = cv2.contourArea(cnt)
        perimeter = cv2.arcLength(cnt, True)
        circularity = (4 * np.pi * area) / (perimeter ** 2) if perimeter > 0 else 0
    else:
        area, perimeter, circularity = 0, 0, 0

    return [contrast, energy, homogeneity, area, perimeter, circularity]

# Klasifikasi Menggunakan RandomForest Classifier

In [66]:
#Klasifikasi data menggunakan RandomForest
def classify_images(features, labels):
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
    classifier = RandomForestClassifier(n_estimators=100, random_state=42)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)

    #Metrik Evaluasi
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    return accuracy, precision, recall, f1

# Main Program

In [67]:
dataset_path = "/content/drive/MyDrive/Cocoa Ripeness Dataset"
categories = ['mentah', 'matang', 'terlalu_matang']

features = []
labels = []
# Pre-process and extract features untuk semua citra gambar
for category in categories:
    category_path = os.path.join(dataset_path, category)
    label = categories.index(category)

    for image_name in os.listdir(category_path):
        image_path = os.path.join(category_path, image_name)
        preprocessed_image = preprocess_image_manual(image_path)
        feature_vector = extract_features(preprocessed_image)
        features.append(feature_vector)
        labels.append(label)

In [68]:
#Klasifikasi dan evaluasi
accuracy, precision, recall, f1 = classify_images(features, labels)

In [69]:
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1-Score: {f1 * 100:.2f}%")

Accuracy: 83.33%
Precision: 79.59%
Recall: 83.33%
F1-Score: 80.01%


# Eksperimen 1

-Noise Reduction menggunakan gaussian blur (sebelumnya mean filter)

 -Penerapan augmentasi rotasi, flipping, dan perubahan brightness.

In [70]:
import random

In [71]:
#Membuat gaussian kernel secara manual
def gaussian_kernel(size, sigma=1):
    kernel = np.zeros((size, size), dtype=np.float32)
    center = size // 2
    for i in range(size):
        for j in range(size):
            diff = (i - center) ** 2 + (j - center) ** 2
            kernel[i, j] = np.exp(-diff / (2 * sigma ** 2))
    kernel /= np.sum(kernel)
    return kernel

In [72]:
#Noise reduction menggunakan gaussian blur
def apply_gaussian_blur(image, kernel_size=3, sigma=1):
    kernel = gaussian_kernel(kernel_size, sigma)
    padded_image = np.pad(image, kernel_size // 2, mode='reflect')
    blurred_image = np.zeros_like(image)

    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            region = padded_image[i:i+kernel_size, j:j+kernel_size]
            blurred_image[i, j] = np.sum(region * kernel)

    return blurred_image.astype(np.uint8)

In [73]:
#Melakukan augmentasi data dengan rotasi, flipping, dan perubahan brightness
def augment_image(image):
    augmented_images = []

    #Rotasi 90 derajat
    rotated_90 = np.rot90(image)
    augmented_images.append(rotated_90)

    #Rotasi 180 derajat
    rotated_180 = np.rot90(image, k=2)
    augmented_images.append(rotated_180)

    #Flipping horizontal
    flipped_horizontal = np.flip(image, axis=1)
    augmented_images.append(flipped_horizontal)

    #Flipping vertical
    flipped_vertical = np.flip(image, axis=0)
    augmented_images.append(flipped_vertical)

    #Perubahan brightness
    brightened = np.clip(image + random.randint(20, 50), 0, 255)
    darkened = np.clip(image - random.randint(20, 50), 0, 255)
    augmented_images.extend([brightened, darkened])

    return augmented_images

In [74]:
#Pipeline pre-processing gambar dengan augmentasi dan gaussian
def preprocess_image_manual(image_path):
    #Membaca gambar menggunakan Pillow untuk mendapatkan dimensi
    with Image.open(image_path) as img:
        img = img.resize((224, 224))  # Resize gambar ke (224, 224) jika diperlukan
        image = np.array(img)  # Konversi ke numpy array (RGB)

    #Pastikan gambar memiliki 3 channel
    if image.ndim == 2:  # Jika grayscale
        image = np.stack([image] * 3, axis=-1)
    elif image.shape[2] == 4:  # Jika RGBA
        image = image[:, :, :3]

    #Pre-processing manual
    grayscale = convert_to_grayscale(image)
    noise_reduced = apply_gaussian_blur(grayscale, kernel_size=5, sigma=1.5)
    enhanced = histogram_equalization(noise_reduced)

    #Data augmentasi
    augmented_images = augment_image(enhanced)

    #Kembalikan semua gambar augmented
    return augmented_images

In [75]:
dataset_path = "/content/drive/MyDrive/Cocoa Ripeness Dataset"
categories = ['mentah', 'matang', 'terlalu_matang']

features = []
labels = []
#Pre-process and extract features untuk semua citra
for category in categories:
    category_path = os.path.join(dataset_path, category)
    label = categories.index(category)

    for image_name in os.listdir(category_path):
        image_path = os.path.join(category_path, image_name)
        preprocessed_image = preprocess_image_manual(image_path)

        #Preprocessing dan augmentasi
        augmented_images = preprocess_image_manual(image_path)

        #Ekstraksi fitur untuk setiap gambar augmented
        for augmented_image in augmented_images:
            feature_vector = extract_features(augmented_image)
            features.append(feature_vector)
            labels.append(label)

In [76]:
accuracy, precision, recall, f1 = classify_images(features, labels)

In [77]:
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1-Score: {f1 * 100:.2f}%")

Accuracy: 90.21%
Precision: 90.21%
Recall: 90.21%
F1-Score: 88.59%


# Eksperimen 2

Melakukan tuning hyperparamter pada model menggunakan GridSearchCV

In [78]:
from sklearn.model_selection import GridSearchCV

In [79]:
#Hyperparameter tuning menggunakan GridSearchCV
def hyperparameter_tuning(features, labels):
    #Split data (80% training, 20% testing)
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

    #Parameter grid untuk GridSearchCV
    param_grid = {
        'n_estimators': [50, 100, 200],  # Jumlah pohon
        'max_depth': [None, 10, 20, 30],  # Kedalaman maksimum
        'min_samples_split': [2, 5, 10],  # Minimum sampel untuk split
        'min_samples_leaf': [1, 2, 4],  # Minimum sampel di leaf node
        'max_features': ['sqrt', 'log2', None]  # Jumlah fitur untuk split
    }

    #Random Forest Classifier
    rf = RandomForestClassifier(random_state=42)

    #Grid Search dengan cross-validation
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy', verbose=2, n_jobs=-1)
    grid_search.fit(X_train, y_train)

    #Best Parameters
    print("Best Parameters:", grid_search.best_params_)

    #Evaluate model dengan parameter terbaik
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)

    #Metrik Evaluasi
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    return accuracy, precision, recall, f1, grid_search.best_params_

In [81]:
dataset_path = "/content/drive/MyDrive/Cocoa Ripeness Dataset"
categories = ['mentah', 'matang', 'terlalu_matang']

features = []
labels = []
#Pre-process and extract features untuk semua citra
for category in categories:
    category_path = os.path.join(dataset_path, category)
    label = categories.index(category)

    for image_name in os.listdir(category_path):
        image_path = os.path.join(category_path, image_name)

        #Preprocessing dan augmentasi
        augmented_images = preprocess_image_manual(image_path)

        #Ekstraksi fitur untuk setiap gambar augmented
        for augmented_image in augmented_images:
            feature_vector = extract_features(augmented_image)
            features.append(feature_vector)
            labels.append(label)

In [82]:
#Konversi fitur dan label ke numpy array
features, labels = np.array(features), np.array(labels)

In [83]:
#Hyperparameter Tuning and Evaluasi
accuracy, precision, recall, f1, best_params = hyperparameter_tuning(features, labels)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Best Parameters: {'max_depth': None, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}


In [84]:
print(f"\nHasil Evaluasi Model Setelah Hyperparameter Tuning:")
print(f"Best Parameters: {best_params}")
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1-Score: {f1 * 100:.2f}%")


Hasil Evaluasi Model Setelah Hyperparameter Tuning:
Best Parameters: {'max_depth': None, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
Accuracy: 91.96%
Precision: 91.83%
Recall: 91.96%
F1-Score: 91.05%
